# 🏆 Final Training Notebook
## Train, evaluate, and save your best model

This notebook brings everything together for the final training run.

## 1. Setup

In [ ]:
import sys
sys.path.append('..')

import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Import your modules
from config import *
from src.utils import set_seed, check_gpu

# Setup
set_seed()
check_gpu()
print_config()

## 2. Load Data

In [ ]:
# Import dataset functions
from src.dataset import ArabicLetterDataset, get_transforms, create_data_loaders

# Get transforms
transform = get_transforms()

# Check if real data exists
csv_path = os.path.join('..', DATA_DIR, "labels.csv")
images_dir = os.path.join('..', DATA_DIR, "images")

if os.path.exists(csv_path):
    # Load real dataset
    full_dataset = ArabicLetterDataset(
        csv_file=csv_path,
        root_dir=images_dir,
        transform=transform
    )

    # Split into train and validation (80/20)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size]
    )

    # Create DataLoaders
    train_loader, val_loader = create_data_loaders(train_dataset, val_dataset)

    print(f"✓ Training samples: {len(train_dataset)}")
    print(f"✓ Validation samples: {len(val_dataset)}")
else:
    # Create dummy data for testing
    print("⚠️ Real data not found. Creating dummy data for testing...")

    num_train, num_val = 500, 100
    train_images = torch.randn(num_train, 1, IMAGE_SIZE, IMAGE_SIZE)
    train_labels = torch.randint(0, NUM_CLASSES, (num_train,))
    val_images = torch.randn(num_val, 1, IMAGE_SIZE, IMAGE_SIZE)
    val_labels = torch.randint(0, NUM_CLASSES, (num_val,))

    train_dataset = TensorDataset(train_images, train_labels)
    val_dataset = TensorDataset(val_images, val_labels)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"✓ Created {num_train} dummy training samples")
    print(f"✓ Created {num_val} dummy validation samples")

## 3. Create Model

In [ ]:
# Import and create model
from src.model import ArabicCNN

model = ArabicCNN().to(DEVICE)

# Print model info
print(f"Model: ArabicCNN")
print(f"Input: (1, {IMAGE_SIZE}, {IMAGE_SIZE}) - Grayscale {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Output: {NUM_CLASSES} classes")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Train

In [ ]:
# Train the model
from src.train import train_model

history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    device=DEVICE
)

## 5. Plot Training History

In [ ]:
# Plot training history
from src.evaluate import plot_training_history

plot_training_history(history, save_path='../outputs/training_history.png')

## 6. Evaluate

In [ ]:
# Evaluate model
from src.evaluate import (
    evaluate_model,
    plot_confusion_matrix,
    get_classification_report,
    get_per_class_accuracy
)

# Get predictions
predictions, labels, accuracy = evaluate_model(model, val_loader, DEVICE)

# Per-class accuracy
get_per_class_accuracy(labels, predictions)

# Classification report
get_classification_report(labels, predictions, save_path='../outputs/classification_report.txt')

# Confusion matrix
plot_confusion_matrix(labels, predictions, save_path='../outputs/confusion_matrix.png')

## 7. Save Final Model

In [ ]:
# Save the final model
from src.train import save_model

save_model(model, '../models/final_model.pth')
print("✓ Model saved to models/final_model.pth")

## 8. Test on Your Own Images!

Try writing an Arabic letter and testing it!

In [ ]:
# Visualize some predictions
from src.evaluate import visualize_predictions

visualize_predictions(model, val_loader, num_images=16, device=DEVICE, save_path='../outputs/prediction_samples.png')

print("\n🎉 Training complete! Check the outputs/ folder for results.")

---

# 🎉 Congratulations!

You've built an AI that can recognize handwritten Arabic letters!

**What you learned:**
- How to load and preprocess image data
- How to build a Convolutional Neural Network
- How to train a model with PyTorch
- How to evaluate model performance

**Next challenges:**
- Can you get accuracy above 90%?
- Try adding more layers or data augmentation
- Build a simple web interface to demo your model